In [1]:
# Import pandas
import pandas as pd 

In [2]:
# Load the data
data = pd.read_csv('https://gitlab.com/AnalyticsIndiaMagazine/practicedatasets/-/raw/main/bootcamp/recommender_system/movies_metadata.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
# Top 5 rows
data.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [4]:
# Shape of the data
data.shape

(45466, 24)

In [5]:
data = data.iloc[:30000,:]

#**Preprocess and define input features**

In [6]:
# Selecting the features 
features = data[['original_title','overview']]

In [7]:
# Check for null values
features.isna().sum()

original_title      0
overview          327
dtype: int64

In [8]:
# Drop null values
features.dropna(inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


#**Vectorizing the features and Computing the similarity between movies**

In [9]:
# Import textvectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Vectorize the input features 
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')

# Get vectorized features
matrix = tf.fit_transform(features['overview'])
matrix.data

array([0.11155648, 0.11155648, 0.11155648, ..., 0.03983817, 0.0341915 ,
       0.05877571])

In [10]:
# import linear kernel
from sklearn.metrics.pairwise import linear_kernel

# get the similarities 
cosine_similarities = linear_kernel(matrix,matrix)

#**Retrieve the results**

In [11]:
# Reset the index
movie_title = features['original_title']
indices = pd.Series(features.index, index=features['original_title'])

In [12]:
# Function get recommendation
def movie_recommend(original_title):

    idx = indices[original_title]

    sim_scores = list(enumerate(cosine_similarities[idx]))

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:31]

    movie_indices = [i[0] for i in sim_scores]

    movie_score = [i[1] for i in sim_scores]
    
    return pd.DataFrame(data = {'Movie names' :list(movie_title.iloc[movie_indices]),'Similarity score' :movie_score})

In [13]:
# Test 1
movie_recommend('The Godfather').head()

,Movie names,Similarity score
0,Portrait in Black,0.089930
1,Hysteria,0.070746
2,Broadway Damage,0.062020
3,Morsian yllättää,0.059723
4,Too Many Husbands,0.053957


In [14]:
# Test 2
movie_recommend('Hulk').head()

,Movie names,Similarity score
0,Black Robe,0.037835
1,Somebody to Love,0.030155
2,Make It Happen,0.026763
3,Cuore sacro,0.025815
4,Hav Plenty,0.025467


In [15]:
# Test 3
movie_recommend('Jumanji').head()

,Movie names,Similarity score
0,Word Wars,0.048527
1,Table No. 21,0.047088
2,Quintet,0.044350
3,The Dark Angel,0.043897
4,Le Pont du Nord,0.043406
